In [1]:

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
# from keras.layers.core import Dense
# from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import matplotlib
import random
import pickle
import cv2
import os

# Parameters:-
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

# Input image:-
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Input_data")))
random.seed(42)
random.shuffle(imagePaths)

# Create an list:-
data=[]
labels=[]

# loop over the input images
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
  

	l = label = imagePath.split(os.path.sep)[-2].split('_')
 
	labels.append(l)

# Convert into numpy both input & lables:-
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# binarizer implementation
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)


#  loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))

# Split Training & Testing
(trainX, testX, trainY, testY) = train_test_split(data,labels,test_size=0.2, random_state=42)

# # LENET ARCHITECTURE CODE

# initialize the model
model = Sequential()

# Rows & Columns  
imgRows=IMAGE_DIMS[0]
imgCols=IMAGE_DIMS[1]
numChannels=IMAGE_DIMS[2]
numClasses=3
inputShape = (imgRows, imgCols, numChannels)

activation="relu"
weightsPath=None

# define the first set of CONV => ACTIVATION => POOL layers
model.add(Conv2D(20, 5, padding="same",input_shape=inputShape))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# define the second set of CONV => ACTIVATION => POOL layers
model.add(Conv2D(50, 5, padding="same"))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# define the first FC => ACTIVATION layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation(activation))

# define the second FC layer
model.add(Dense(numClasses))


# lastly, define the soft-max classifier
model.add(Activation("softmax"))

# if a weights path is supplied (inicating that the model was
# pre-trained), then load the weights
if weightsPath is not None:
  model.load_weights(weightsPath)

#compile 
model.compile(loss = "CategoricalCrossentropy",optimizer = 'adam',metrics = ['accuracy'])

# # fitting the model 
hist = model.fit(x=trainX,y=trainY,epochs = 11,batch_size = 9,validation_data =(testX,testY),verbose = 1)

# evaluate the model
test_score = model.evaluate(testX,testY)
print("Test loss {:.5f},accuracy {:.3f}".format(test_score[0],test_score[1]*100))

# Save the model
# model.save('TRAINING_EXPERIENCE.h5')

# f = open("mlb.pickle", "wb")
# f.write(pickle.dumps(mlb))
# f.close()
# print(trainX.shape)
# print(testX.shape)

# print(trainY.shape)
# print(testY.shape)

ImportError: ignored

In [ ]:
# OWN DATA SET IMAGE CLASSIFICATION USING LENET-ARCHITECTURE MODEL:-


#Importing keras libraries and packages
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import matplotlib
import numpy as np
import cv2
import pickle
import imutils

# Load Model:-
model = load_model("/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Model/TRAINING_EXPERIENCE.h5")
mlb = pickle.loads(open("/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Model/mlb.pickle","rb").read())

# Read an Input image:-
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Dataset/critical/16.jpg')
output = imutils.resize(image,width=400)
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
proba = model.predict(image)[0]
print(proba)
idxs = np.argsort(proba)[::-1][:2]
print(idxs)

for (i, j) in enumerate(idxs):
	label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
	cv2.putText(output, label, (10, (i * 30) + 25), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
 
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Sample_output/good.jpg',output)


[9.9985266e-01 9.6038035e-05 5.1240411e-05]
[0 1]


True

New Train code

In [7]:
import matplotlib
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from ARCHITECTURE.lenet import LeNet
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os

# Parameters:-
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

# Input image:-
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Dataset")))
print(imagePaths)

random.seed(42)
random.shuffle(imagePaths)

# Create an list:-
data=[]
labels=[]

# loop over the input images
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)

	l = label = imagePath.split(os.path.sep)[-2].split("_")
	labels.append(l)

# Convert into numpy both input & lables:-
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# binarizer implementation
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

#  loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))

# Split Training & Testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

# Model:-
model = LeNet.build(numChannels=IMAGE_DIMS[2],imgRows=IMAGE_DIMS[0], imgCols=IMAGE_DIMS[1],numClasses=2,activation="relu",weightsPath=None)

#compile 
model.compile(loss = keras.losses.categorical_crossentropy,optimizer = 'SGD',metrics = ['accuracy'])

# fitting the model 
hist = model.fit(x=trainX,y=trainY,epochs = 50,batch_size = 128,validation_data =(testX,testY),verbose = 1)

# evaluate the model
test_score = model.evaluate(testX,testY)
print("Test loss {:.5f},accuracy {:.3f}".format(test_score[0],test_score[1]*100))

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Model/TRAINING_EXPERIENCE.h5')
f = open("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Model/mlb.pickle", "wb")
f.write(pickle.dumps(mlb))
f.close()



ModuleNotFoundError: ignored

In [6]:
!pip install libarchive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 46.6 MB/s 
  ERROR: Failed building wheel for libarchive
  Running setup.py clean for libarchive
Failed to build libarchive
    Running setup.py install for libarchive ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-y2373hwx/libarchive_c27f0d5e2bcb4799bd5ecf8350bd28c8/setup.py'"'"'; __file__='"'"'/tmp/pip-install-y2373hwx/libarchive_c27f0d5e2bcb4799bd5ecf8350bd28c8/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-otjp31g9/install-record.txt --single-version-externally-managed --com

In [ ]:
# OWN DATA SET IMAGE CLASSIFICATION USING LENET-ARCHITECTURE MODEL:-


#Importing keras libraries and packages
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import pickle
import imutils

# Load Model:-
model = load_model("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Model/TRAINING_EXPERIENCE.h5")
mlb = pickle.loads(open("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Model/mlb.pickle", "rb").read())

# Read an Input image:-
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Input_data/good_Road/2.jpg')
output = imutils.resize(image,width=400)
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
proba = model.predict(image)[0]
print(proba)
idxs = np.argsort(proba)[::-1][:2]
print(idxs)

for (i, j) in enumerate(idxs):
	label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
	print(mlb.classes_[j])
	print(label)
	cv2.putText(output, label, (10, (i * 30) + 25), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_Image_Classification/Input_data/out1.jpg',output)

In [ ]:
import matplotlib
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from ARCHITECTURE.lenet import LeNet
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os

ImportError: ignored

In [ ]:
! python3 -c 'import keras; print(keras.__version__)' // 2.4.3 

2.7.0
